In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,accuracy_score

import matplotlib.pyplot as plt

In [ ]:
iris=load_iris()

X=iris.data
y=iris.target.reshape(-1,1)

print(X.shape,y.shape)

(150, 4) (150, 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


In [ ]:
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [ ]:
class ModelDNN(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim,64 ),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # 3 output classes
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
input_dim = X_train.shape[1] # 4 features for Iris
num_classes = len(torch.unique(y_train)) # 3 classes for Iris

class ModelDNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ModelDNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.model(x)

model = ModelDNN(input_dim, num_classes)

In [ ]:
model = ModelDNN(input_dim=X_train.shape[1], num_classes=num_classes)
print(model)

ModelDNN(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=3, bias=True)
  )
)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [32]:
epochs=50
train_losses=[]

for epoch in range(epochs):
  model.train()
  train_loss=0.0
  epoch_loss=0.0
  for X_batch,y_batch in train_loader:
    optimizer.zero_grad()
    predictions=model(X_batch)
    # Fix: Squeeze the target tensor to make it 1D
    loss=criterion(predictions,y_batch.squeeze(1))
    loss.backward()
    optimizer.step()
    epoch_loss +=loss.item()

  epoch_loss=epoch_loss/len(train_loader)
  avg_loss=epoch_loss/len(train_loader)
  train_losses.append(avg_loss)

  if (epoch+1)%10==0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

Epoch [10/50], Loss: 0.0003
Epoch [20/50], Loss: 0.0002
Epoch [30/50], Loss: 0.0002
Epoch [40/50], Loss: 0.0001
Epoch [50/50], Loss: 0.0001


In [33]:
X_test_tensor = X_test
y_test_tensor = y_test

In [34]:
from sklearn.metrics import accuracy_score

with torch.no_grad():
  # Get logits from the model for the test set
  outputs = model(X_test_tensor)
  # Get the predicted class by finding the index with the maximum logit
  _, y_pred_labels = torch.max(outputs, 1)

# Convert y_pred and y_test to numpy for sklearn metrics
y_pred_np = y_pred_labels.numpy()

y_true_np = y_test_tensor.squeeze(1).numpy()


acc = accuracy_score(y_true_np, y_pred_np)

print(f'Accuracy Score: {acc:.4f}')

Accuracy Score: 0.9667


In [35]:
torch.save(model.state_dict(), 'iris_regression_model.pth')
print('Model saved as iris_regression_model.pth')

Model saved as iris_regression_model.pth
